# Autocombat TOP+StrokeMRI, SABREand Insight46 harmonized datasets

This notebook creates datasets from the above hamornized as if each of 1. a blended TOP and StrokeMRI 2.SABRE and 3. Insight  is indpendant and from a different "batch" (TOP and StrokeMRI are same acquisition on same machine, so same batch)

Note this must be run in the `comscan` ( `comscan6`) environment

## import libraries

In [ ]:
import os

import pandas as pd
import numpy as np

In [ ]:
import ComScan

## Let's see how we would apply this to our data
We will have to flip it on it's side to make it work

In [ ]:
!python --version

In [ ]:
filepath = '../open_work/internal_results/cleaned_pvc2s/' 
filename_mri = os.path.join(filepath,'StrokeMRI_pvc2c.csv') 
filename_top = os.path.join(filepath,'TOP_pvc2c.csv') 
filename_sabre = os.path.join(filepath,'SABRE_pvc2_cleaned.csv') 
filename_insight46 =  os.path.join(filepath,'Insight46_pvc2c.csv') 

In [ ]:
# Now we must clean SABRE down to be like the other datasets, including dropping NAns
MRI = pd.read_csv(filename_mri)
TOP = pd.read_csv(filename_top)
Insight46 = pd.read_csv(filename_insight46)

SABRE =  pd.read_csv(filename_sabre)
coly = TOP.columns
# SABRE = SABRE[coly]
# SABRE= SABRE.dropna()
SABRE.tail(3)

In [ ]:
TOP['site'] = 0
MRI['site'] = 1
MRI.head(3)

# Here we will harmonize just TOP and StrokeMRI

In [ ]:
TOPMRI = pd.concat([TOP, MRI])
TOPMRI.head(3) 

In [ ]:
sex_mapping = {'F':0,'M':1}
TOPMRI = TOPMRI.assign(sex = TOPMRI.sex.map(sex_mapping))
TOPMRI.head(2) 

## Now very importantly, we have our sites pre-defined. This means that the AutoCOmbat module will execute it's own version of combat (without clustering). Therefore we can implement the Autocombat neurocombat module's combat, without a need for clustering
If you don't believe me just read the documentation and the code itself

classComScan.neurocombat.Combat(features, sites, discrete_covariates=None, continuous_covariates=None, ref_site=None, empirical_bayes=True, parametric=True, mean_only=False, return_only_features=False, raise_ref_site=True, copy=True)

In [ ]:
TOPMRI['decade']=(TOPMRI['age']/10).round()
#TOPMRI['decade']

In [ ]:
TOPMRI = TOPMRI.sort_values(by='age')
TOPMRI.reset_index(inplace=True)
TOPMRI['fine_grain'] = TOPMRI['age'].rolling(2).sum()/2
#TOPMRI

In [ ]:
TOPMRI[2:].fine_grain.iloc[::2] = TOPMRI[:].fine_grain.iloc[1::2]
#TOPMRI['fine_grain']

In [ ]:
TOPMRI['fine_grain'][0] = TOPMRI['fine_grain'][1]
TOPMRI['fine_grain']

In [ ]:
TOPMRI['decade'].unique()

In [ ]:
combat = ComScan.neurocombat.Combat(features=['gm_vol', 'wm_vol',
       'csf_vol', 'gm_ivc_ratio', 'gmwm_ivc_ratio', 'wmh_vol', 'wmh_count',
       'deepwm_b_cov', 'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
       'deepwm_b', 'aca_b', 'mca_b', 'pca_b', 'totalgm_b',], sites=[ "site"], discrete_covariates=['sex'],continuous_covariates=['decade'],)

fg_combat = ComScan.neurocombat.Combat(features=['gm_vol', 'wm_vol',
       'csf_vol', 'gm_ivc_ratio', 'gmwm_ivc_ratio', 'wmh_vol', 'wmh_count',
       'deepwm_b_cov', 'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
       'deepwm_b', 'aca_b', 'mca_b', 'pca_b', 'totalgm_b',], sites=[ "site"], discrete_covariates=['sex'],continuous_covariates=['fine_grain'],)


In [ ]:
print(combat.fit(TOPMRI))
print(fg_combat.fit(TOPMRI))

In [ ]:
print(combat.gamma_star_)
print(fg_combat.gamma_star_)

In [ ]:
#print(combat.transform(TOPMRI))

In [ ]:
TOPMRI.head(3)

In [ ]:
transformed_TOPMRI= combat.transform(TOPMRI)
transformed_TOPMRI.head(3)

In [ ]:
fg_transformed_TOPMRI= fg_combat.transform(TOPMRI)
fg_transformed_TOPMRI.head(3)

In [ ]:
transformed_TOPMRI.columns

In [ ]:
TOP_transformed = transformed_TOPMRI[ transformed_TOPMRI['site']==0]
TOP_transformed = TOP_transformed.drop(['site', 'decade', 'Unnamed: 0','fine_grain', 'index'], axis=1)
TOP_transformed.head(3)

In [ ]:
fg_TOP_transformed = fg_transformed_TOPMRI[ fg_transformed_TOPMRI['site']==0]
fg_TOP_transformed = fg_TOP_transformed.drop(['site', 'decade', 'Unnamed: 0','fine_grain', 'index'], axis=1)
fg_TOP_transformed.head(3)

In [ ]:
MRI_transformed = transformed_TOPMRI[ transformed_TOPMRI['site']==1]
MRI_transformed = MRI_transformed.drop(['site', 'decade', 'Unnamed: 0', 'index', 'fine_grain'], axis=1)
MRI_transformed.head(3)

In [ ]:
fg_MRI_transformed = fg_transformed_TOPMRI[fg_transformed_TOPMRI['site']==1]
fg_MRI_transformed = fg_MRI_transformed.drop(['site', 'decade', 'Unnamed: 0', 'index', 'fine_grain'], axis=1)
fg_MRI_transformed.head(3)

In [ ]:
TOP_transformed.to_csv('../open_work/internal_results/autocombat/autocom_harm_top1.csv')
MRI_transformed.to_csv('../open_work/internal_results/autocombat/autocom_harm_mri1.csv')

fg_TOP_transformed.to_csv('../open_work/internal_results/autocombat/fg_autocom_harm_top1.csv')
fg_MRI_transformed.to_csv('../open_work/internal_results/autocombat/fg_autocom_harm_mri1.csv')

# Now we will harmonize TOPMRI as one dataset agains sabre

In [ ]:
TOPMRI4 = TOPMRI.copy()
TOPMRI4['site'] = 0
TOPMRI4.site.unique()

In [ ]:
TOPMRI4.columns

In [ ]:
SABRE['site'] = 1

In [ ]:
#SABRE.isna().sum()

In [ ]:
SABRE['decade']=(SABRE['age']/10).round()
#SABRE['decade']

In [ ]:
SABRE = SABRE.assign(sex = SABRE.sex.map(sex_mapping))
SABRE.head(3) 

In [ ]:
TOPMRI4SABRE =pd.concat([TOPMRI4, SABRE])

In [ ]:
TOPMRI4SABRE = TOPMRI4SABRE.sort_values(by='age')
TOPMRI4SABRE.reset_index(inplace=True)
TOPMRI4SABRE['fine_grain'] = TOPMRI4SABRE['age'].rolling(2).sum()/2
TOPMRI4SABRE.head(5)

In [ ]:
TOPMRI4SABRE[:].fine_grain.iloc[::2] = TOPMRI4SABRE[:].fine_grain.iloc[1::2]
TOPMRI4SABRE['fine_grain']

In [ ]:
print(combat.fit(TOPMRI4SABRE))
print(fg_combat.fit(TOPMRI4SABRE))

In [ ]:
print(combat.gamma_star_)
print(fg_combat.gamma_star_)


In [ ]:
transformed_TOPMRI4SABRE= combat.transform(TOPMRI4SABRE)
transformed_TOPMRI4SABRE.head(3)

In [ ]:
fg_transformed_TOPMRI4SABRE= fg_combat.transform(TOPMRI4SABRE)
fg_transformed_TOPMRI4SABRE.head(3)

In [ ]:
TOPMRI4_transformed = transformed_TOPMRI4SABRE[ transformed_TOPMRI4SABRE['site']==0]
TOPMRI4_transformed = TOPMRI4_transformed.drop(['site', 'decade', 'Unnamed: 0', 'fine_grain', 'index', 'level_0'], axis=1)
TOPMRI4_transformed.head(2)

In [ ]:
fg_TOPMRI4_transformed = fg_transformed_TOPMRI4SABRE[fg_transformed_TOPMRI4SABRE['site']==0]
fg_TOPMRI4_transformed = fg_TOPMRI4_transformed.drop(['site', 'decade', 'Unnamed: 0', 'fine_grain', 'index','level_0'], axis=1)
fg_TOPMRI4_transformed.head(3)

In [ ]:
SABRE_transformed = transformed_TOPMRI4SABRE[ transformed_TOPMRI4SABRE['site']==1]
SABRE_transformed = SABRE_transformed.drop(['site', 'decade', 'Unnamed: 0', 'fine_grain', 'index', 'level_0'], axis=1)
SABRE_transformed.head(2)

In [ ]:
fg_SABRE_transformed = fg_transformed_TOPMRI4SABRE[fg_transformed_TOPMRI4SABRE['site']==1]
fg_SABRE_transformed = fg_SABRE_transformed.drop(['site', 'decade', 'Unnamed: 0', 'fine_grain', 'index', 'level_0'], axis=1)
fg_SABRE_transformed.head(3)

In [ ]:
TOPMRI4_transformed.to_csv('../open_work/internal_results/autocombat/autocom_harm_top_and_mri_against_sabre.csv')
SABRE_transformed.to_csv('../open_work/internal_results/autocombat/autocom_harm_sabre_against_top_and_mri.csv')

fg_TOPMRI4_transformed.to_csv('../open_work/internal_results/autocombat/fg_autocom_harm_top_and_mri_against_sabre.csv')
fg_SABRE_transformed.to_csv('../open_work/internal_results/autocombat/fg_autocom_harm_sabre_against_top_and_mri.csv')

# Now the same with TOPMRI and Inisght46

In [ ]:
Insight46['site'] = 1

In [ ]:
Insight46.isna().sum()

In [ ]:
Insight46['decade']=(Insight46['age']/10).round()
Insight46['decade']

In [ ]:
Insight46 = Insight46.assign(sex = Insight46.sex.map(sex_mapping))
Insight46.head(3) 

In [ ]:
Insight46.head(3)

In [ ]:
TOPMRI4Insight =pd.concat([TOPMRI4, Insight46])

In [ ]:
TOPMRI4Insight = TOPMRI4Insight.sort_values(by='age')
TOPMRI4Insight.reset_index(inplace=True)
TOPMRI4Insight['fine_grain'] = TOPMRI4Insight['age'].rolling(2).sum()/2
TOPMRI4Insight.head(3)

In [ ]:
TOPMRI4Insight[2:].fine_grain.iloc[::2] = TOPMRI4Insight[:].fine_grain.iloc[1::2]
TOPMRI4Insight['fine_grain']

In [ ]:
TOPMRI4Insight['fine_grain'][0] = TOPMRI4Insight['fine_grain'][1]
TOPMRI4Insight['fine_grain']

In [ ]:
print(combat.fit(TOPMRI4Insight))
print(fg_combat.fit(TOPMRI4Insight))

In [ ]:
print(combat.gamma_star_)
print(fg_combat.gamma_star_)


In [ ]:
transformed_TOPMRI4Insight= combat.transform(TOPMRI4Insight)
transformed_TOPMRI4Insight.head(3)

In [ ]:
fg_transformed_TOPMRI4Insight= fg_combat.transform(TOPMRI4Insight)
fg_transformed_TOPMRI4Insight.head(3)

In [ ]:
TOPMRI4_transformed = transformed_TOPMRI4Insight[ transformed_TOPMRI4Insight['site']==0]
TOPMRI4_transformed = TOPMRI4_transformed.drop(['site', 'decade', 'Unnamed: 0', 'fine_grain', 'index', 'level_0'], axis=1)
TOPMRI4_transformed.head(2)

In [ ]:
fg_TOPMRI4_transformed = fg_transformed_TOPMRI4Insight[fg_transformed_TOPMRI4Insight['site']==0]
fg_TOPMRI4_transformed = fg_TOPMRI4_transformed.drop(['site', 'decade', 'Unnamed: 0', 'fine_grain', 'index', 'level_0'], axis=1)
fg_TOPMRI4_transformed.head(3)

In [ ]:
Insight_transformed = transformed_TOPMRI4Insight[ transformed_TOPMRI4Insight['site']==1]
Insight_transformed = Insight_transformed.drop(['site', 'decade', 'Unnamed: 0', 'fine_grain', 'index', 'level_0'], axis=1)
Insight_transformed.head(2)

In [ ]:
fg_Insight_transformed = fg_transformed_TOPMRI4Insight[fg_transformed_TOPMRI4Insight['site']==1]
fg_Insight_transformed = fg_Insight_transformed.drop(['site', 'decade', 'Unnamed: 0', 'fine_grain', 'index', 'level_0'], axis=1)
fg_Insight_transformed.head(3)

In [ ]:
TOPMRI4_transformed.to_csv('../open_work/internal_results/autocombat/autocom_harm_top_and_mri_against_insight.csv')
Insight_transformed.to_csv('../open_work/internal_results/autocombat/autocom_harm_insight_against_top_and_mri.csv')

fg_TOPMRI4_transformed.to_csv('../open_work/internal_results/autocombat/fg_autocom_harm_top_and_mri_against_insight.csv')
fg_Insight_transformed.to_csv('../open_work/internal_results/autocombat/fg_autocom_harm_insight_against_top_and_mri.csv')

# Now we harmonize TOPMRI against both SABRE and Insight

In [ ]:
Insight46['site'] = 3

In [ ]:
SABRE['site'] = 4

In [ ]:
TOPMRI6ALL =pd.concat([TOPMRI4, Insight46, SABRE])

In [ ]:
TOPMRI6ALL = TOPMRI6ALL.sort_values(by='age')
TOPMRI6ALL.reset_index(inplace=True)
TOPMRI6ALL['fine_grain'] = TOPMRI6ALL['age'].rolling(2).sum()/2
TOPMRI6ALL.head(3)

In [ ]:
TOPMRI6ALL[:].fine_grain.iloc[::2] = TOPMRI6ALL[:].fine_grain.iloc[1::2]
TOPMRI6ALL['fine_grain']

In [ ]:
print(combat.fit(TOPMRI6ALL))
print(fg_combat.fit(TOPMRI6ALL))

In [ ]:
print(combat.gamma_star_)
print(fg_combat.gamma_star_)

In [ ]:
transformed_TOPMRI6ALL= combat.transform(TOPMRI6ALL)
transformed_TOPMRI6ALL.head(3)

In [ ]:
fg_transformed_TOPMRI6ALL= fg_combat.transform(TOPMRI6ALL)
fg_transformed_TOPMRI6ALL.head(3)

In [ ]:
TOPMRI6_transformed =  transformed_TOPMRI6ALL[ transformed_TOPMRI6ALL['site']==0]
TOPMRI6_transformed.head(2) 

In [ ]:
TOPMRI6_transformed.columns

In [ ]:
fg_TOPMRI6_transformed =  fg_transformed_TOPMRI6ALL[ fg_transformed_TOPMRI6ALL['site']==0]
fg_TOPMRI6_transformed.head(2) 

In [ ]:
TOPMRI6_transformed =TOPMRI6_transformed.drop(['site', 'decade', 'Unnamed: 0', 'fine_grain', 'index','level_0'], axis=1)
TOPMRI6_transformed.head(2)

In [ ]:
fg_TOPMRI6_transformed =fg_TOPMRI6_transformed.drop(['site', 'decade', 'Unnamed: 0', 'fine_grain', 'index','level_0'], axis=1)
fg_TOPMRI6_transformed.head(2)

In [ ]:
Inisght2_transformed = transformed_TOPMRI6ALL[ transformed_TOPMRI6ALL['site']==3]
Inisght2_transformed = Inisght2_transformed.drop(['site', 'decade', 'Unnamed: 0', 'index', 'fine_grain', 'level_0'], axis=1)
Inisght2_transformed.head(2)

In [ ]:
fg_Inisght2_transformed = fg_transformed_TOPMRI6ALL[fg_transformed_TOPMRI6ALL['site']==3]
fg_Inisght2_transformed = fg_Inisght2_transformed.drop(['site', 'decade', 'Unnamed: 0', 'index', 'fine_grain', 'level_0'], axis=1)
fg_Inisght2_transformed.head(2)

In [ ]:
SABRE2_transformed = transformed_TOPMRI6ALL[ transformed_TOPMRI6ALL['site']==4]
SABRE2_transformed = SABRE2_transformed.drop(['site', 'decade', 'Unnamed: 0', 'fine_grain', 'index', 'level_0'], axis=1)
SABRE2_transformed.head(3)

In [ ]:
fg_SABRE2_transformed = fg_transformed_TOPMRI6ALL[fg_transformed_TOPMRI6ALL['site']==4]
fg_SABRE2_transformed = fg_SABRE2_transformed.drop(['site', 'decade', 'Unnamed: 0', 'fine_grain', 'index', 'level_0'], axis=1)
fg_SABRE2_transformed.head(3)

In [ ]:
TOPMRI6_transformed.to_csv('../open_work/internal_results/autocombat/autocom_harm_top_and_mri_against_inisght_w_sabre.csv')
Inisght2_transformed.to_csv('../open_work/internal_results/autocombat/autocom_harm_insight_against_top_and_mri_w_sabre.csv')
SABRE2_transformed.to_csv('../open_work/internal_results/autocombat/autocom_harm_sabret_against_top_and_mri_w_insight.csv')

In [ ]:
fg_TOPMRI6_transformed.to_csv('../open_work/internal_results/autocombat/fg_autocom_harm_top_and_mri_against_inisght_w_sabre.csv')
fg_Inisght2_transformed.to_csv('../open_work/internal_results/autocombat/fg_autocom_harm_insight_against_top_and_mri_w_sabre.csv')
fg_SABRE2_transformed.to_csv('../open_work/internal_results/autocombat/fg_autocom_harm_sabret_against_top_and_mri_w_insight.csv')